# LOG-02: Forecasting & Inventory Policies — Answer Key

In [ ]:
import pandas as pd, numpy as np
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

df = pd.read_csv("../data/daily_demand.csv", parse_dates=["date"]).sort_values("date")
for lag in [1,7,14]:
    df[f"lag{lag}"] = df["demand"].shift(lag)
df = df.dropna().reset_index(drop=True)
split = int(len(df)*0.8)
train, test = df.iloc[:split], df.iloc[split:]
X_tr = train[["lag1","lag7","lag14"]].values
y_tr = train["demand"].values
X_te = test[["lag1","lag7","lag14"]].values
y_te = test["demand"].values

rf = RandomForestRegressor(random_state=42).fit(X_tr, y_tr)
test["forecast"] = rf.predict(X_te)

# EOQ/ROP/SS
K=80; h=1.2; lead=5; z=1.65
annual_demand = df["demand"].sum()
EOQ = ((2*K*annual_demand)/h)**0.5
demand_std = train["demand"].rolling(30).std().dropna().mean()
mu_LT = test["forecast"].rolling(lead).sum().fillna(test["forecast"].mean()*lead)
sigma_LT = (lead**0.5)*demand_std
SS = z*sigma_LT
ROP = mu_LT + SS

print("EOQ:", round(EOQ,2))
out = pd.DataFrame({"date": test["date"], "forecast": test["forecast"], "ROP": ROP})
print(out.head())

out.plot(x="date", y=["forecast","ROP"])
plt.title("Forecast & Reorder Point")
plt.show()
